In [31]:
# Install necessary libraries
!pip install nltk requests

In [32]:
# Import modules
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import requests
from IPython.display import display, HTML

In [33]:
# Download NLTK resources
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [34]:

def detect_emotion(user_input):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(user_input)
    sentiment_score = sentiment['compound']

    if sentiment_score >= 0.05:
        return 'positive', sentiment_score
    elif sentiment_score <= -0.05:
        return 'negative', sentiment_score
    else:
        return 'neutral', sentiment_score

In [35]:
def get_movies_by_emotion(emotion, api_key):
    genre_map = {
        'positive': 35,  # Comedy
        'negative': 18,  # Drama
        'neutral': 28    # Action
    }

    genre_id = genre_map.get(emotion, 28)  # Default to Action if emotion is unknown
    url = f"https://api.themoviedb.org/3/discover/movie?api_key={api_key}&with_genres={genre_id}&language=en-US&page=1"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        movies = response.json().get('results', [])
        if not movies:
            return ["No movies found for this emotion."]

        movie_details = []
        for movie in movies:
            title = movie['title']
            overview = movie.get('overview', 'No description available.')
            movie_details.append(f"<b>{title}</b>: {overview}")
        return movie_details
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from TMDB API: {e}")
        return ["Error fetching movie data. Please try again later."]

In [36]:
def format_output(emotion, sentiment_score, movies):
    color = {
        'positive': 'green',
        'negative': 'red',
        'neutral': 'blue'
    }.get(emotion, 'black')

    message = {
        'positive': "😊 You seem happy! How about enjoying some comedy?",
        'negative': "😢 It looks like you're feeling down. Maybe a drama might help you feel better.",
        'neutral': "😐 Feeling neutral? An action-packed movie could be just what you need."
    }.get(emotion, "Here's a movie recommendation for you:")

    formatted_movies = "<br>".join(movies)

    html_content = f"""
    <div style="font-family: Arial, sans-serif; color: {color};">
        <h2>{message}</h2>
        <p><b>Detected Emotion:</b> {emotion.capitalize()} (Sentiment Score: {sentiment_score:.2f})</p>
        <h3>Recommended Movies:</h3>
        <div>{formatted_movies}</div>
    </div>
    """
    display(HTML(html_content))

In [38]:
def main():
    # Replace with your TMDB API key
    API_KEY = '93bc7635258e9ae1252debd5b671e1eb'

    # Get user input
    user_input = input("How are you feeling today? ")

    # Check for empty input
    if not user_input.strip():
        print("You didn't enter anything. Please provide a valid input.")
        return

    # Detect emotion and sentiment score
    emotion, sentiment_score = detect_emotion(user_input)

    # Get movie recommendations
    movies = get_movies_by_emotion(emotion, API_KEY)

    # Format and display output
    format_output(emotion, sentiment_score, movies)

if __name__ == "__main__":
    main()

How are you feeling today? thriller
